# Problem 4 Solution

I chose to do problem 4 from the list of screener questions. I didn't look at the qiskit implementation, except to compare the output of my code and that of qiskit.

## Qiskit Circuit
To validate my implementation, first we create a simple circuit in qiskit and run simluation on it.

In [1]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(5, 5)
qc.h(0)
qc.h(1)
qc.cx(1, 0)
qc.cy(3, 1)
qc.cz(0, 4)
qc.h(3)
qc.unitary([[0.5 + 0.5j, 0.5 - 0.5j], [0.5 - 0.5j, 0.5 + 0.5j]], 2)
out = qc.measure([0, 1, 2, 3, 4], [0, 1, 2, 3, 4])
qc.draw()


┌───┐   ┌───┐             ┌─┐      
q_0: ───┤ H ├───┤ X ├──────■──────┤M├──────
        ├───┤   └─┬─┘┌───┐ │      └╥┘┌─┐   
q_1: ───┤ H ├─────■──┤ Y ├─┼───────╫─┤M├───
     ┌──┴───┴──┐ ┌─┐ └─┬─┘ │       ║ └╥┘   
q_2: ┤ unitary ├─┤M├───┼───┼───────╫──╫────
     └─────────┘ └╥┘   │   │ ┌───┐ ║  ║ ┌─┐
q_3: ─────────────╫────■───┼─┤ H ├─╫──╫─┤M├
                  ║        │ └┬─┬┘ ║  ║ └╥┘
q_4: ─────────────╫────────■──┤M├──╫──╫──╫─
                  ║           └╥┘  ║  ║  ║ 
c: 5/═════════════╩════════════╩═══╩══╩══╩═
                  2            4   0  1  3

In [2]:
from qiskit import Aer, execute

# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = execute(qc, backend_sim, shots=10000)

# Grab the results from the job.
result_sim = job_sim.result()
qiskit_counts = result_sim.get_counts(qc)

## JQiskit Circuit
Then I built the same circuit in my simulator and ran it.

In [3]:
from jqiskit.api import QuantumCircuit as JQuantumCircuit

qc = JQuantumCircuit(5)
qc.h(0)
qc.h(1)
qc.cx(1, 0)
qc.cy(3, 1)
qc.cz(0, 4)
qc.h(3)
qc.sqrtnot(2)
print(qc)
jqiskit_counts, state = qc.measure(num_shots=10000)

Hadamard: (0,)
Hadamard: (1,)
CX: (1, 0)
CY: (3, 1)
CZ: (0, 4)
Hadamard: (3,)
SQRTNOT: (2,)


## Validation

I then compared the two by looking at the simulation output states. Note how qiskit is the opposite endian-ness as my implementation, so I had to flip the state to do a proper comparison.

In [4]:
assert len(jqiskit_counts) == len(qiskit_counts), "Number of states don't match!"

for state_str in jqiskit_counts:
    print(f'state: {state_str}, qiskit: {qiskit_counts[state_str[::-1]]}; jqiskit: {jqiskit_counts[state_str]}')

state: 00100, qiskit: 621; jqiskit: 620
state: 01000, qiskit: 621; jqiskit: 597
state: 10000, qiskit: 634; jqiskit: 610
state: 00110, qiskit: 622; jqiskit: 672
state: 00010, qiskit: 656; jqiskit: 655
state: 11010, qiskit: 635; jqiskit: 647
state: 01110, qiskit: 604; jqiskit: 590
state: 01100, qiskit: 605; jqiskit: 624
state: 11000, qiskit: 624; jqiskit: 647
state: 11110, qiskit: 619; jqiskit: 578
state: 11100, qiskit: 635; jqiskit: 609
state: 10110, qiskit: 589; jqiskit: 589
state: 10100, qiskit: 628; jqiskit: 628
state: 00000, qiskit: 648; jqiskit: 657
state: 01010, qiskit: 625; jqiskit: 620
state: 10010, qiskit: 634; jqiskit: 657


In [5]:
import numpy as np

qc = JQuantumCircuit(1)
qc.parametric('[[1j + 0.4, 2], [3j ,4 + sin(theta)]]', 0)
qc.measure(feed_dict={'theta': np.pi/4})

({'0': 221, '1': 779}, array([0.4+1.j, 2. +0.j]))